# Solving WORDLE with Information Theory

This is the companion notebook of this [post](https://jaclx5.github.io/wardle) with the same title.

(add a pointer to the wordle solution game).

## Seting up the stage

In [1]:
import copy
import math
import re

from functools import reduce
from collections import Counter

from tqdm import tqdm

In [2]:
WORDS_FILE = "words05.txt"

ALL_LETTERS = list(map(chr, range(65, 91))) # all letters A-Z

Here we'll use the list of 5 letters extracted from the Ubuntu English dictionary:

In [3]:
words = open(WORDS_FILE).read().strip().split("\n")

## Most Informative Letters

The first question we want to answer is: "What is the most informative letter"?

Let's start by counting the occurence of each letter in each individual positions.

In [4]:
N = len(words)

cmap = []
imap = []

# loop over the 5 positions
for i in range(5):
    cchar = {}

    # loop over the letters of the alphabet
    for c in ALL_LETTERS:
        (g, y, x) = (0, 0, 0)
        
        # loop over all words
        # and count the number of times letter `c`...
        for w in words:
            if w[i] == c:
                # ...appears in position i-th
                g += 1
            elif c in w:
                # ...appears in the word
                y += 1
            else:
                # ...does not appear in the word
                x += 1
        
        # store the counts for letter `c`
        cchar[c] = {'g': g, 'y': y, 'x': x}
    
    # store the counts for position `i`
    cmap.append(cchar)

Now we can ask, for example, how many times:

"A" and "Z" appear in the 1st position:

In [5]:
cmap[0]['A']['g'], cmap[0]['Z']['g']

(228, 13)

"A" and "Z" appear in the word but not in the 1st position:

In [6]:
cmap[0]['A']['y'], cmap[0]['Z']['y']

(1508, 75)

In [7]:
# "A" and "Z" do not appear in the word
cmap[0]['A']['x'], cmap[0]['Z']['x'], 

(2858, 4506)

Now, that we have counts, wehave everyting to compute the probabilities and the quantity of information of each letter in each position.

In [8]:
inf_map = []

for i, v in enumerate(cmap):
    inf = {}
    
    for c, counts in v.items():
        # probability of each of the outcomes for letter `c` at position `i`
        pg, py, px = (counts['g']+1)/(N+1), (counts['y']+1)/(N+1), (counts['x']+1)/(N+1)
        
        # quantity of information
        inf[c] = -(pg * math.log2(pg) + py * math.log2(py) + px * math.log2(px))
        
    inf_map.append(inf)

Note that in the probability formula above, instead of simply computing:

$P(g) = {g_{count} \over N}$

we use instead:

$P(g) = {{g_{count} + 1} \over {N + 1}}$

This is a called smooting and it's a technical trick to avoid probabilities 0 which would make our life miserabel when computing logarithms. The final values will be slightly different but the overall result will not change.

We could have computed both quantities at the same time while counting the letters, but I find it more pedagogic this way.

So, what it the most (and the least) informative letter (MIL) for each position?

In [9]:
print("The Most Informative Letters are:")

for i in range(5):   
    c = max(inf_map[i], key=inf_map[i].get)
    print(f"Pos {i+1}: '{c}' with a {inf_map[i][c]:3.2f} bits of information.")

The Most Informative Letters are:
Pos 1: 'S' with a 1.42 bits of information.
Pos 2: 'E' with a 1.37 bits of information.
Pos 3: 'A' with a 1.29 bits of information.
Pos 4: 'E' with a 1.46 bits of information.
Pos 5: 'S' with a 1.45 bits of information.


In [10]:
print("The Least Informative Letters are:")

for i in range(5):   
    c = min(inf_map[i], key=inf_map[i].get)
    print(f"Pos {i+1}: '{c}' with a {inf_map[i][c]:3.2f} bits of information.")

The Least Informative Letters are:
Pos 1: 'Q' with a 0.09 bits of information.
Pos 2: 'Q' with a 0.08 bits of information.
Pos 3: 'Q' with a 0.08 bits of information.
Pos 4: 'Q' with a 0.08 bits of information.
Pos 5: 'Q' with a 0.08 bits of information.


Poor letter "_Q_"!

# Which Word Maximizes the Individual Letter Information?

We can sum the quantity of information of all letter of a word and look for the word that maximizes that value:

In [11]:
max_inf = 0
max_word = ""

for w in words:
    # sum the quantity of information of each letter of the word `w`
    inf = sum([inf_map[i][c] for i, c in enumerate(w)])
    
    if inf > max_inf:
        max_inf, max_word = inf, w

And the word is:

In [12]:
max_word

'SAREE'

Letters are not independent:

In [13]:
def occurences(seq, words):
    print(f"'{seq}' occurs {sum(map(lambda w: w.count(seq), words))} times in all words.")

occurences("E", words)
occurences("T", words)

'E' occurs 2443 times in all words.
'T' occurs 1266 times in all words.


Knowing thar we find almost twice more "E" than "T" in all words.

In [14]:
occurences("AE", words)

'AE' occurs 12 times in all words.


And that we find 12 "E" after "A", we would expect to find (by independence) 6 "T" after "A":

In [15]:
occurences("AT", words)

'AT' occurs 129 times in all words.


In fact we find 129, slightly more than 6, which points strongly against the independence of letter occurence, which is, by the way, common knowledge!

# Most Informative Word

Lets extend the Most Informative Letter approach the find the Most Informative Word.

First we need a function to compute the response clue for any given pair of solution and guess word:

In [16]:
def compute_response(solution, guess):
    resp = ""

    for cw, cg in zip(solution, guess):
        if cw == cg:
            resp += "g"
        elif cg in solution:
            resp += "y"
        else:
            resp += "x"

    return resp

compute_response("VIDEO", "OLDEN")

'yxggx'

Next we'll compute the quantity of information of a given guess word by:
1. Compute all possible occurrences of response clues for that guess word.
1. Counting how many times each response occurs for that word.
1. Use this value to compute the probability of the response and the quantity of information.

In [17]:
def word_information(guess, words):
    responses = []
    
    # Compute all possible occurrences of response clues.
    for solution in words:
        responses.append(compute_response(solution, guess))

    # Count how many times each response occurs
    inf = 0
    for count_response in Counter(responses).values():
        # Compute the probability of the response and the quantity of information.
        inf += -math.log2(count_response/N) * (count_response/N)
    
    return inf

For example, $I(DRINK)$ is:

In [18]:
word_information("DRINK", words)

4.492051185462364

Finally we just have to repeat the same process for all words and find the MIW:

In [19]:
def most_informative_word(words):
    max_inf = 0
    max_word = ""

    # for all the guess words
    for guess in tqdm(words):
        inf = word_information(guess, words)
        
        if inf > max_inf:
            max_inf = inf
            max_word = guess
    
    return max_word, max_inf

And the MIW is:

In [20]:
most_informative_word(words)

100%|███████████████████████████████████████████████████████████████| 4594/4594 [00:11<00:00, 389.71it/s]


('TARES', 6.217111890623548)

In [21]:
def least_informative_word(words):
    min_inf = 10**10
    min_word = ""

    # for all the guess words
    for guess in tqdm(words):
        inf = word_information(guess, words)
        
        if inf < min_inf:
            min_inf = inf
            min_word = guess
    
    return min_word, min_inf

Just for sake of curiosity the LIW is:

In [22]:
least_informative_word(words)

100%|███████████████████████████████████████████████████████████████| 4594/4594 [00:11<00:00, 389.77it/s]


('FUZZY', 2.1844580656481196)

In [23]:
word_information("FUZZY", words)

2.1844580656481196

# Let's Play a Game

We'll assume the hidden solution word:

~~~
VIDEO
~~~

Starting with:

~~~
TARES
~~~

We obtain:

In [24]:
compute_response("VIDEO", "TARES")

'xxxgx'

We can build a simple regex to filter all words not compatible with this clue:

In [25]:
def select(words, regex, yellows=""):
    # test:
    # 1. if all letters in yellows are in the word
    # 2. if the word match the regex
    return list(filter(lambda w: not (set(yellows) - set(w)) and re.match(regex, w), words))

The filtered list will help us find the next MIW:

In [26]:
words2 = select(words, r"[^ARST][^ARST][^ARST]E[^ARST]")

In [27]:
most_informative_word(words2)

100%|████████████████████████████████████████████████████████████████| 176/176 [00:00<00:00, 7960.63it/s]


('OLDEN', 0.33586599340984086)

In [28]:
compute_response("VIDEO", "OLDEN")

'yxggx'

We repeat the same strategy:

In [29]:
words3 = select(words2, r"[^ALNORST][^ALNRST]DE[^ALNRST]", "O")
most_informative_word(words3)

100%|███████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 58743.75it/s]


('CODED', 0.013240678212209322)

In [30]:
compute_response("VIDEO", "CODED")

'xyggy'

And again...

In [31]:
words4 = select(words3, r"[^ACDLNORST][^ACDLNORST]DE[^ACDLNRST]", "OD")
most_informative_word(words4)

100%|███████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 20460.02it/s]


('VIDEO', 0.0026481356424418643)

In [32]:
compute_response("VIDEO", "VIDEO")

'ggggg'

Tcham!!

# More 